In [2]:
!pip install ffmpeg-python pillow opencv-python
!git clone https://github.com/soCzech/TransNetV2.git
%cd TransNetV2/inference

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
fatal: destination path 'TransNetV2' already exists and is not an empty directory.
/home/daoan/Projects/AI_Challenge_HCMC_2024/data_extraction/frame_split/TransNetV2/inference


/home/daoan/Projects/AI_Challenge_HCMC_2024/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


'hello.'

In [1]:
%cd TransNetV2/inference
%pwd

/home/daoan/Projects/AI_Challenge_HCMC_2024/data_extraction/frame_split/TransNetV2/inference


/home/daoan/Projects/AI_Challenge_HCMC_2024/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


'/home/daoan/Projects/AI_Challenge_HCMC_2024/data_extraction/frame_split/TransNetV2/inference'

In [2]:
# Import module
import os
import cv2
import json
import glob
import ffmpeg
import numpy as np
from tqdm import tqdm
from transnetv2 import TransNetV2

2024-10-12 14:53:20.334434: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-12 14:53:20.370037: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-12 14:53:20.381296: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-12 14:53:20.383824: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-12 14:53:20.399562: I tensorflow/core/platform/cpu_feature_guar

In [3]:
import tensorflow as tf
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

2.17.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


I0000 00:00:1728719603.725067 2639220 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728719603.851631 2639220 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728719603.851728 2639220 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


In [4]:
videos_dir = '/media/daoan/T7 Shield2/AI_Challenge_2024_DATA/video_with_audio'
all_video_paths = dict()
for part in sorted(os.listdir(videos_dir)):
    data_part = part.split('_')[-1] # L01, L02 for ex
    if int(data_part.replace('L', '')) >= 25: 
        all_video_paths[data_part] =  dict()

for data_part in sorted(all_video_paths.keys()):
    data_part_path = f'{videos_dir}/Videos_{data_part}/video'
    video_paths = sorted(os.listdir(data_part_path))
    video_ids = [video_path.replace('.mp4', '').split('_')[-1] for video_path in video_paths]
    for video_id, video_path in zip(video_ids, video_paths):
        video_path_full = f'{data_part_path}/{video_path}'
        all_video_paths[data_part][video_id] = video_path_full

In [5]:
model = TransNetV2()

save_dir = './SceneJSON_2024'
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

for key, video_paths_dict in tqdm(all_video_paths.items()):
    video_ids = sorted(video_paths_dict.keys())

    if not os.path.exists(os.path.join(save_dir, key)):
        os.mkdir(os.path.join(save_dir, key))

    for video_id in tqdm(video_ids):
        video_path = video_paths_dict[video_id]
        _, single_frame_predictions, _ = model.predict_video(video_path)

        # Generate list of scenes from predictions, returns tuples of (start frame, end frame)
        scenes = model.predictions_to_scenes(single_frame_predictions)
        print(f"{save_dir}/{key}/{video_id}.json")
        with open(f"{save_dir}/{key}/{video_id}.json", 'w') as f:
            json.dump(scenes.tolist(), f)

[TransNetV2] Using weights from /home/daoan/Projects/AI_Challenge_HCMC_2024/data_extraction/frame_split/TransNetV2/inference/transnetv2-weights/.


I0000 00:00:1728719640.286651 2639220 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728719640.286769 2639220 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728719640.286812 2639220 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728719640.336155 2639220 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

[TransNetV2] Extracting frames from /media/daoan/T7 Shield2/AI_Challenge_2024_DATA/video_with_audio/Videos_L25/video/L25_V001.mp4


2024-10-12 14:54:14.308730: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
W0000 00:00:1728719654.457003 2639690 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1728719654.519876 2639690 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1728719654.521730 2639690 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1728719654.535702 2639690 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1728719654.535888 2639690 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1728719654.536071 2639690 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1728719654.592813 2639690 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1728719654.594304 2639690 gpu_t

[TransNetV2] Processing video frames 39900/40064

...

[TransNetV2] Processing video frames 5477/5477
./SceneJSON_2024/L30/V094.json
[TransNetV2] Extracting frames from /media/daoan/T7 Shield2/AI_Challenge_2024_DATA/video_with_audio/Videos_L30/video/L30_V095.mp4
[TransNetV2] Processing video frames 7500/7739


 99%|█████████▉| 95/96 [03:48<00:02,  2.88s/it]

[TransNetV2] Processing video frames 7739/7739
./SceneJSON_2024/L30/V095.json
[TransNetV2] Extracting frames from /media/daoan/T7 Shield2/AI_Challenge_2024_DATA/video_with_audio/Videos_L30/video/L30_V096.mp4
[TransNetV2] Processing video frames 1850/2336


100%|██████████| 6/6 [1:07:04<00:00, 670.68s/it]

[TransNetV2] Processing video frames 2336/2336
./SceneJSON_2024/L30/V096.json


In [1]:
import torch
torch.backends.cudnn.version()



8907

AttributeError: module 'tensorflow.python.platform.build_info' has no attribute 'cudnn_version_number'